In [ ]:
import urllib.request
import pandas as pd
import urllib
import bs4 as bs
from bs4 import SoupStrainer
from bs4 import BeautifulSoup
import urllib.request, urllib.error, urllib.parse
import time
from urllib.request import FancyURLopener  # This is library that helps us create the headless browser
import random
from random import choice  #This library helps pick a random item from a list

In [ ]:
import time
import pandas as pd
from urllib.request import FancyURLopener  # This is library that helps us create the headless browser
import random
from random import choice #This library helps pick a random item from a list

# create user agents
user_agents = [
    'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
    'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16',
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/7046A194A',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
]

# read csv file
df = pd.read_csv("homeneeds.csv", encoding = 'utf-8')

# create product links
link = []
for i in df['product_link']:
    index = i.find("ref=")
    i = i[:index]
    link.append("https://www.amazon.com" + i)

# create review page link
rating = []
for i in df['rating_link']:
    index = i.find("ref=")
    i = i[:index]
    rating.append("https://www.amazon.com" + i)    

# define class for fancy url opener
class MyOpener(FancyURLopener, object):
    version = choice(user_agents)

a = 1 # variable for denoting rank
rank = [] # list for storing rank
review = [] # list for storing review
num_images = [] # list for storing number of images

for i in link: # iterate over all product links
    
    try:
        myopener = MyOpener()
        page=myopener.open(i) # open page using fancy url
        time.sleep(random.uniform(0,2))  #sleep time 

        html = page.read().decode('utf-8') # generate html for the page
        soup = bs.BeautifulSoup(html) # create bs obejct for html source code

        # get number of images
        infotable = soup.find_all("li", class_ = "a-spacing-small item")
        num_images.append(len(infotable)) # will have length 100 for 100 products
        
    except:
        num_images.append('no images')

    # get link for review page
    url = [] # list to capture link for review page
    infotable = soup.find_all("a", class_ = "a-link-emphasis a-text-bold")
    if infotable: # if link for review page is found
        for link in infotable:
            url.append("https://www.amazon.com" + link.get('href')) #create url for the review page
        try:
            # run the below code if review page opens
            myopener = MyOpener()
            page=myopener.open(url[0]) # open review page
            time.sleep(random.uniform(0,2))  #sleep time 
            html = page.read().decode('utf-8')
            soup = bs.BeautifulSoup(html)

            # get top 10 reviews
            infotable = soup.find_all("div", class_ = "a-row a-spacing-small review-data")
            for link in infotable:
                review.append(link.getText()) # append each review
                rank.append(a) #append rank for each review
        except:
            # run if review page does not open
            rank.append(a)
            review.append("no review")
    else: # if link for review page is not found
        rank.append(a)
        review.append("no review")
    
    a = a+1 # go to next rank

In [ ]:
# writing the reviews back to a csv file
import pandas as pd
import csv
df = pd.DataFrame(list(zip(rank, review)), 
            columns =['rank', 'review'])

df.to_csv('/Users/ruthvik/Downloads/homeneeds_rating.csv', index = False)

In [ ]:
# writing the images back to a csv file
df = pd.DataFrame(list(zip(num_images)), 
            columns =['num_images'])
df.to_csv('/Users/ruthvik/Downloads/homeneeds_images.csv', index = False)